In [8]:
import os
import numpy as np
import pandas as pd
from collections import OrderedDict
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms, models, datasets
from PIL import Image

In [9]:
TRAIN_DIR = os.path.join('../data/train')
VALID_DIR = os.path.join('../data/valid')
BATCH_SIZE = 10
IMAGE_SIZE = 100

In [10]:
transformer = transforms.Compose([transforms.Resize(IMAGE_SIZE),
                                  transforms.RandomRotation(degrees=20),
                                  transforms.ToTensor()])
train_set = torchvision.datasets.ImageFolder(root=TRAIN_DIR, transform=transformer)
valid_set = torchvision.datasets.ImageFolder(root=VALID_DIR, transform=transformer)

In [11]:
train_loader = DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False
)

valid_loader = DataLoader(
    valid_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False
)

In [12]:
squeezenet = models.squeezenet1_0(num_classes=4)
squeezenet.features[0] = nn.Conv2d(3, 100, kernel_size=(3, 3), stride=(2, 2))
squeezenet.features[3].squeeze = nn.Conv2d(100, 16, kernel_size=(1, 1), stride=(1, 1))
squeezenet

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 100, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(100, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
 

In [13]:
def trainer(model, criterion, optimizer, trainloader, validloader, epochs=5):

    train_loss = []
    valid_loss = []
    train_acc = []
    valid_acc = []

    for epoch in range(epochs):

        train_batch_loss = 0
        valid_batch_loss = 0
        train_batch_acc = 0
        valid_batch_acc= 0

        for X, y in trainloader:

            optimizer.zero_grad()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            train_batch_loss += loss.item()
            y_label = y_pred.argmax(dim=1)
            train_batch_acc += sum(y == y_label).type(torch.float32) / len(y)

        train_loss.append(train_batch_loss / len(trainloader))
        train_acc.append(train_batch_acc / len(trainloader))

        model.eval()
        with torch.no_grad():
            for X_valid, y_valid in validloader:
                y_pred = model(X_valid)
                loss = criterion(y_pred, y_valid)
                y_label = y_pred.argmax(dim=1)
                valid_batch_loss += loss.item()
                valid_batch_acc += sum(y_valid == y_label).double() / len(y_valid)

            valid_loss.append(valid_batch_loss / len(validloader))
            valid_acc.append(valid_batch_acc / len(validloader))
        model.train()
        
        

        print(f"""Epoch #{epoch+1}, Train loss: {train_loss[-1]:.2f}, Train Acc: {train_acc[-1]:.2f},
          Valid loss: {valid_loss[-1]:.2f}, Valid Acc: {valid_acc[-1]:.2f}""")
    return train_acc, valid_acc

In [14]:
CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = optim.Adam(squeezenet.parameters(), lr=0.0001)

In [8]:
trainer(squeezenet, CRITERION, OPTIMIZER, train_loader, valid_loader, epochs=15)

Epoch #1, Train loss: 1.35, Train Acc: 0.26,
          Valid loss: 1.38, Valid Acc: 0.35
Epoch #2, Train loss: 1.01, Train Acc: 0.50,
          Valid loss: 0.93, Valid Acc: 0.51
Epoch #3, Train loss: 0.88, Train Acc: 0.58,
          Valid loss: 1.03, Valid Acc: 0.51
Epoch #4, Train loss: 1.02, Train Acc: 0.50,
          Valid loss: 1.03, Valid Acc: 0.49
Epoch #5, Train loss: 1.03, Train Acc: 0.50,
          Valid loss: 1.04, Valid Acc: 0.49
Epoch #6, Train loss: 1.02, Train Acc: 0.50,
          Valid loss: 1.02, Valid Acc: 0.49
Epoch #7, Train loss: 1.01, Train Acc: 0.51,
          Valid loss: 0.99, Valid Acc: 0.51
Epoch #8, Train loss: 0.95, Train Acc: 0.52,
          Valid loss: 0.80, Valid Acc: 0.68
Epoch #9, Train loss: 0.57, Train Acc: 0.69,
          Valid loss: 0.54, Valid Acc: 0.77
Epoch #10, Train loss: 0.20, Train Acc: 0.92,
          Valid loss: 0.34, Valid Acc: 0.83
Epoch #11, Train loss: 0.02, Train Acc: 0.99,
          Valid loss: 0.02, Valid Acc: 0.99
Epoch #12, Train lo

In [9]:
MODEL_NAME = "model.pt"
model_save_path = f"../model/{MODEL_NAME}"
torch.save(squeezenet.state_dict(), model_save_path)